# Data Cleaning and Data Processing - Gan Khai Li

In this phase, we will perform data cleaning on the data in order to remove missing values, duplicates and non-alphebetic words.

In [1]:
#Import necessary libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, when, lower, col
from pyspark.sql.types import StringType
import re

#Initialize Spark session
spark = SparkSession.builder.appName("Data_Cleaning_and_Data_Processing").getOrCreate()
sc = spark.sparkContext

24/12/21 17:51:48 WARN Utils: Your hostname, MSI. resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/12/21 17:51:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/21 17:51:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
#Scan the text file and save it into rdd
raw_paragraphs_rdd = sc.textFile("raw_paragraphs.txt")
raw_paragraphs_rdd.collect()

['Keluarga (Jawi: \u200fکلوارݢ\u200ecode: ms is deprecated \u200e) ialah satu kumpulan manusia yang dihubungkan melalui pertalian darah, perkahwinan atau penjagaan di mana para anggotanya tinggal dalam suatu tempat di bawah suatu rumah dalam keadaan saling bergantungan.[1]',
 '',
 'Ia merupakan unit terkecil dalam sesebuah masyarakat yang asasnya terdiri daripada orang tua dan anak; orang tua tersebut ada baik secara berpasangan (sebagai suami isteri) ataupun tunggal - sama ada kerana kematian pasangan (duda atau janda) mahupun perceraian dengan pasangan tersebut.',
 '',
 'Menurut Salvicion dan Celis (1998) di dalam keluarga terdapat dua atau lebih dari dua pribadi yang tergabung kerana hubungan darah, hubungan perkahwinan atau pengangkatan, di hidupnya dalam satu rumah tangga, berinteraksi satu sama lain dan di dalam perannya masing-masing dan menciptakan serta mempertahankan suatu kebudayaan.[2]',
 '',
 'Di Barat, keluarga didefinisikan sebagai satu kumpulan manusia yang mempunyai hu

In [3]:
#Split the paragraphs into single words
def tokenize(text):
    return text.split()

words_rdd = raw_paragraphs_rdd.flatMap(tokenize)
words_rdd.collect()

['Keluarga',
 '(Jawi:',
 '\u200fکلوارݢ\u200ecode:',
 'ms',
 'is',
 'deprecated',
 '\u200e)',
 'ialah',
 'satu',
 'kumpulan',
 'manusia',
 'yang',
 'dihubungkan',
 'melalui',
 'pertalian',
 'darah,',
 'perkahwinan',
 'atau',
 'penjagaan',
 'di',
 'mana',
 'para',
 'anggotanya',
 'tinggal',
 'dalam',
 'suatu',
 'tempat',
 'di',
 'bawah',
 'suatu',
 'rumah',
 'dalam',
 'keadaan',
 'saling',
 'bergantungan.[1]',
 'Ia',
 'merupakan',
 'unit',
 'terkecil',
 'dalam',
 'sesebuah',
 'masyarakat',
 'yang',
 'asasnya',
 'terdiri',
 'daripada',
 'orang',
 'tua',
 'dan',
 'anak;',
 'orang',
 'tua',
 'tersebut',
 'ada',
 'baik',
 'secara',
 'berpasangan',
 '(sebagai',
 'suami',
 'isteri)',
 'ataupun',
 'tunggal',
 '-',
 'sama',
 'ada',
 'kerana',
 'kematian',
 'pasangan',
 '(duda',
 'atau',
 'janda)',
 'mahupun',
 'perceraian',
 'dengan',
 'pasangan',
 'tersebut.',
 'Menurut',
 'Salvicion',
 'dan',
 'Celis',
 '(1998)',
 'di',
 'dalam',
 'keluarga',
 'terdapat',
 'dua',
 'atau',
 'lebih',
 'dari',
 '

In [4]:
words_rdd = words_rdd.map(lambda word: (word,))
words_df = words_rdd.toDF(["Kata"])
words_df.show(10)

+-------------+
|         Kata|
+-------------+
|     Keluarga|
|       (Jawi:|
|‏کلوارݢ‎code:|
|           ms|
|           is|
|   deprecated|
|           ‎)|
|        ialah|
|         satu|
|     kumpulan|
+-------------+
only showing top 10 rows



In [5]:
#Calculate the total number of data
dataCount = words_df.count()
print(f"Total number of rows: {dataCount}")

Total number of rows: 95761


In [6]:
#Declare a function to remove all the non-alphabetic word
def clean_text(text):
    return re.sub(r'[^a-zA-Z]', '', text)

In [7]:
#Remove all the non-alphabetic and assign word "none" into the blanks
clean_text_udf = udf(clean_text, StringType())
words_df = words_df.withColumn("Kata", clean_text_udf(words_df["Kata"]))
words_df = words_df.withColumn(
    "Kata", 
    when(words_df["Kata"] == "", "None").otherwise(words_df["Kata"])
)

words_df.show(10)

+----------+
|      Kata|
+----------+
|  Keluarga|
|      Jawi|
|      code|
|        ms|
|        is|
|deprecated|
|      None|
|     ialah|
|      satu|
|  kumpulan|
+----------+
only showing top 10 rows



In [8]:
#Convert all the words into lowercase
words_df = words_df.withColumn("Kata", lower(words_df["Kata"]))
words_df.show(10)

+----------+
|      Kata|
+----------+
|  keluarga|
|      jawi|
|      code|
|        ms|
|        is|
|deprecated|
|      none|
|     ialah|
|      satu|
|  kumpulan|
+----------+
only showing top 10 rows



In [9]:
#Calculate the total number of data
dataCount = words_df.count()
print(f"Total number of rows: {dataCount}")

Total number of rows: 95761


In [10]:
#Drop all the missing values
cleaned_data = words_df.dropna()
cleaned_data.show(10)

+----------+
|      Kata|
+----------+
|  keluarga|
|      jawi|
|      code|
|        ms|
|        is|
|deprecated|
|      none|
|     ialah|
|      satu|
|  kumpulan|
+----------+
only showing top 10 rows



In [11]:
#Calculate the total number of data after dropping missing values
dataCount = cleaned_data.count()
print(f"Total number of rows: {dataCount}")

Total number of rows: 95761


In [12]:
#Examine the duplicate words
duplicate_words_df = cleaned_data.groupBy("Kata").count().filter(col("count") > 1)
print("Duplicate Words:")
duplicate_words_df.show(10)

Duplicate Words:
+--------+-----+
|    Kata|count|
+--------+-----+
| asasnya|    4|
|semangat|    7|
|generasi|   48|
| familia|    4|
|  merrie|    4|
|  hingga|   46|
|  larang|    4|
|     art|   16|
|  online|   24|
|    some|    8|
+--------+-----+
only showing top 10 rows



In [13]:
#Drop all the duplicate words
cleaned_data = cleaned_data.dropDuplicates(["Kata"])
print("DataFrame after dropping duplicates:")
cleaned_data.show(10)

DataFrame after dropping duplicates:
+--------+
|    Kata|
+--------+
| asasnya|
|semangat|
|generasi|
| familia|
|  merrie|
|  hingga|
|  larang|
|     art|
|  online|
|    some|
+--------+
only showing top 10 rows



In [14]:
cleaned_data = cleaned_data.where(words_df["Kata"] != "na")
cleaned_data.show()

+------------+
|        Kata|
+------------+
|     asasnya|
|    semangat|
|    generasi|
|     familia|
|      merrie|
|      hingga|
|      larang|
|         art|
|      online|
|        some|
|      travel|
|   teknologi|
|   northvegr|
|       sedar|
|    pengamal|
|      kanser|
|dipraktikkan|
|      lapang|
|        guli|
|        paut|
+------------+
only showing top 20 rows



In [15]:
dataCount = cleaned_data.count()
print(f"Total number of rows: {dataCount}")

Total number of rows: 7924


In [16]:
#Arrange the word list into ascending order
words_df = cleaned_data.orderBy("Kata", ascending=True)  
words_df.show()

+--------------+
|          Kata|
+--------------+
|             a|
|            aa|
|           aan|
|         aaron|
|     aaronwooi|
|            ab|
|          abad|
|       abcclio|
|           abd|
|         abdul|
|      abdullah|
|      abkhazia|
|           abm|
|        abodes|
|         about|
|      aboutcom|
|            ac|
|      academic|
|       academy|
|acanthocephala|
+--------------+
only showing top 20 rows



In [17]:
dataCount = words_df.count()
print(f"Total number of rows: {dataCount}")

Total number of rows: 7924


In [18]:
#Save the words into words.csv
words_df = words_df.coalesce(1)

output_csv_path = "cleaned_word.csv"
words_df.write.csv(output_csv_path, header=True)

In [19]:
spark.stop()